In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import RobustScaler, StandardScaler

- Distribution visualization
- Missing value analysis
- Correlation analysis with automatic selection of correlation method
- Smart normalization based on outlier presence
- Outlier visualization
- Comprehensive summary report generation

In [ ]:
class UniversalEDA:
    def __init__(self, df):
        """
        Initialize the EDA class with a DataFrame
        
        Parameters:
        -----------
        df : pandas.DataFrame
            DataFrame to analyze
        """
        self.df = df
        self.numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
        
    def plot_distributions(self, figsize=(15, 5*len(self.numeric_columns))):
        """
        Visualize the distribution of numerical variables
        - Histogram
        - Box plot
        - Q-Q plot for normality check
        """
        n_cols = len(self.numeric_columns)
        
        fig, axes = plt.subplots(n_cols, 3, figsize=figsize)
        
        for idx, col in enumerate(self.numeric_columns):
            # Histogram
            sns.histplot(data=self.df, x=col, ax=axes[idx, 0])
            axes[idx, 0].set_title(f'{col} Distribution')
            
            # Box plot
            sns.boxplot(data=self.df, y=col, ax=axes[idx, 1])
            axes[idx, 1].set_title(f'{col} Boxplot')
            
            # Q-Q plot
            stats.probplot(self.df[col].dropna(), dist="norm", plot=axes[idx, 2])
            axes[idx, 2].set_title(f'{col} Q-Q Plot')
        
        plt.tight_layout()
        return fig
    
    def check_missing_values(self):
        """
        Analyze missing values
        - Count and percentage of missing values for each variable
        - Return rows containing missing values
        """
        missing_count = self.df.isnull().sum()
        missing_percent = (missing_count / len(self.df)) * 100
        missing_summary = pd.DataFrame({
            'Missing Count': missing_count,
            'Missing Percent': missing_percent
        }).sort_values('Missing Count', ascending=False)
        
        print("\n=== Missing Value Summary ===")
        print(missing_summary[missing_summary['Missing Count'] > 0])
        
        print("\n=== Rows with Missing Values ===")
        return self.df[self.df.isnull().any(axis=1)]
    
    def correlation_analysis(self):
        """
        Analyze correlations between variables
        - Use Pearson or Spearman correlation based on normality test
        """
        correlation_matrix = pd.DataFrame(index=self.numeric_columns, columns=self.numeric_columns)
        method_matrix = pd.DataFrame(index=self.numeric_columns, columns=self.numeric_columns)
        
        for col1 in self.numeric_columns:
            for col2 in self.numeric_columns:
                # Shapiro-Wilk test for normality
                _, p_val1 = stats.shapiro(self.df[col1].dropna())
                _, p_val2 = stats.shapiro(self.df[col2].dropna())
                
                # If both variables are normally distributed (p > 0.05), use Pearson
                if p_val1 > 0.05 and p_val2 > 0.05:
                    corr, _ = stats.pearsonr(self.df[col1].dropna(), self.df[col2].dropna())
                    method = 'Pearson'
                else:
                    corr, _ = stats.spearmanr(self.df[col1].dropna(), self.df[col2].dropna())
                    method = 'Spearman'
                
                correlation_matrix.loc[col1, col2] = corr
                method_matrix.loc[col1, col2] = method
        
        # Plot correlation heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Correlation Matrix')
        plt.show()
        
        print("\n=== Correlation Method Used ===")
        print(method_matrix)
        
        return correlation_matrix, method_matrix
    
    def normalize_data(self, columns=None):
        """
        Normalize data using appropriate scaling method
        - RobustScaler for data with outliers
        - StandardScaler for data without outliers
        
        Parameters:
        -----------
        columns : list
            List of columns to normalize (None for all numeric variables)
        
        Returns:
        --------
        normalized_df : pandas.DataFrame
            Normalized DataFrame
        scalers : dict
            Dictionary of scaler objects used for each column
        """
        if columns is None:
            columns = self.numeric_columns
            
        normalized_df = self.df.copy()
        scalers = {}
        
        for col in columns:
            # Check outliers using IQR method
            Q1 = self.df[col].quantile(0.25)
            Q3 = self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            outlier_range = 1.5 * IQR
            outliers = ((self.df[col] < (Q1 - outlier_range)) | 
                       (self.df[col] > (Q3 + outlier_range))).sum()
            
            # Use RobustScaler if outliers are more than 1% of data
            if outliers / len(self.df) >= 0.01:
                scaler = RobustScaler()
                print(f"{col}: Using RobustScaler (Found {outliers} outliers)")
            else:
                scaler = StandardScaler()
                print(f"{col}: Using StandardScaler")
            
            normalized_df[col] = scaler.fit_transform(self.df[[col]])
            scalers[col] = scaler
        
        return normalized_df, scalers
    
    def plot_outliers(self, columns=None):
        """
        Visualize outliers using box plots
        
        Parameters:
        -----------
        columns : list
            List of columns to visualize (None for all numeric variables)
        """
        if columns is None:
            columns = self.numeric_columns
            
        n_cols = len(columns)
        fig, axes = plt.subplots(n_cols, 1, figsize=(10, 5*n_cols))
        if n_cols == 1:
            axes = [axes]
            
        for idx, col in enumerate(columns):
            
            # Calculate outlier bounds
            Q1 = self.df[col].quantile(0.25)
            Q3 = self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Create boxplot
            sns.boxplot(data=self.df, y=col, ax=axes[idx])
            
            # Add text with outlier information
            outliers = ((self.df[col] < lower_bound) | (self.df[col] > upper_bound)).sum()
            axes[idx].set_title(f'{col} Outliers: {outliers} points')
            
        plt.tight_layout()
        return fig

    def generate_summary_report(self):
        """
        Generate comprehensive data summary report
        - Basic information
        - Numeric summary
        - Missing values analysis
        - Distribution plots
        - Correlation analysis
        - Outlier visualization
        """
        print("=== Data Summary Report ===")
        print("\nBasic Information:")
        print(f"- Total Rows: {len(self.df)}")
        print(f"- Total Columns: {len(self.df.columns)}")
        print(f"- Numeric Columns: {len(self.numeric_columns)}")
        print(f"- Non-numeric Columns: {len(self.df.columns) - len(self.numeric_columns)}")
        
        print("\nNumeric Columns Summary:")
        print(self.df[self.numeric_columns].describe())
        
        print("\nMissing Values Summary:")
        self.check_missing_values()
        
        # Generate and save all plots
        self.plot_distributions()
        plt.savefig('distributions.png')
        
        self.correlation_analysis()
        plt.savefig('correlation.png')
        
        self.plot_outliers()
        plt.savefig('outliers.png')
        
        print("\nPlots have been saved as:")
        print("- distributions.png")
        print("- correlation.png")
        print("- outliers.png")

In [ ]:
# Usage Example:
"""
# Initialize the class with a DataFrame
eda = UniversalEDA(df)

# Generate comprehensive summary report
eda.generate_summary_report()

# Or perform specific analyses
eda.plot_distributions()  # Check distributions
eda.check_missing_values()  # Analyze missing values
eda.correlation_analysis()  # Analyze correlations
eda.plot_outliers()  # Visualize outliers

# Normalize data
normalized_df, scalers = eda.normalize_data()
"""

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, spearmanr, pearsonr
from sklearn.preprocessing import StandardScaler, RobustScaler

# 1. Plot distribution of numerical features
def plot_distribution(df):
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        plt.figure(figsize=(8, 4))
        sns.histplot(df[col], kde=True, bins=30, color='blue')
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.show()

# 2. Missing Value Analysis
def missing_value_analysis(df):
    missing_counts = df.isnull().sum()
    missing_counts = missing_counts[missing_counts > 0]
    print("Missing Values Per Column:")
    print(missing_counts)
    
    if missing_counts.sum() > 0:
        print("\nRows with Missing Values:")
        display(df[df.isnull().any(axis=1)])

# 3. Compute Correlation (Pearson if normal, Spearman otherwise)
def calculate_correlation(df):
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    correlation_matrix = pd.DataFrame(index=num_cols, columns=num_cols)
    
    for col1 in num_cols:
        for col2 in num_cols:
            if col1 != col2:
                stat, p = shapiro(df[col1].dropna())
                if p > 0.05:  # Normally distributed
                    corr, _ = pearsonr(df[col1].dropna(), df[col2].dropna())
                else:  # Not normally distributed
                    corr, _ = spearmanr(df[col1].dropna(), df[col2].dropna())
                correlation_matrix.loc[col1, col2] = corr
    
    correlation_matrix = correlation_matrix.astype(float)
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Correlation Matrix")
    plt.show()
    
    return correlation_matrix

# 4. Detect Outliers
def detect_outliers(df, method="zscore", threshold=3):
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    outlier_dict = {}
    
    for col in num_cols:
        if method == "zscore":
            mean, std = df[col].mean(), df[col].std()
            z_scores = (df[col] - mean) / std
            outliers = df[np.abs(z_scores) > threshold]
        elif method == "iqr":
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
        
        if not outliers.empty:
            outlier_dict[col] = outliers
    
    return outlier_dict

# 5. Normalize Data
def normalize_data(df):
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    scaler_dict = {}
    df_scaled = df.copy()
    
    for col in num_cols:
        outliers = detect_outliers(df[[col]], method="zscore", threshold=3)
        
        if len(outliers) > 0:
            scaler = RobustScaler()
            print(f"Using RobustScaler for {col} (outliers detected)")
        else:
            scaler = StandardScaler()
            print(f"Using StandardScaler for {col} (no outliers detected)")
        
        df_scaled[col] = scaler.fit_transform(df[[col]])
        scaler_dict[col] = scaler
    
    return df_scaled, scaler_dict


In [ ]:
df[column_name] = df[column_name].fillna(df[column_name].rolling(window, min_periods=1).mean())

In [ ]:
###Time-Based Features:
# Basic time features
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['day_of_week'] = df['Date'].dt.dayofweek
df['week_of_year'] = df['Date'].dt.isocalendar().week
df['quarter'] = df['Date'].dt.quarter

# Seasonal features
df['is_winter'] = df['month'].isin([12, 1, 2])
df['is_summer'] = df['month'].isin([6, 7, 8])
df['season'] = pd.cut(df['month'], 
                     bins=[0, 2, 5, 8, 11, 12], 
                     labels=['Winter', 'Spring', 'Summer', 'Fall', 'Winter'])

# Holiday features
from holidays import US
holidays_us = US()
df['is_holiday'] = df['Date'].isin(holidays_us)
df['is_weekend'] = df['day_of_week'].isin([5, 6])

###Temperature-Related Features:
# Temperature variations
df['temp_diff'] = df['actual_temp'] - df['forecasted_average_temp']
df['temp_lag_1d'] = df['actual_temp'].shift(1)
df['temp_lag_2d'] = df['actual_temp'].shift(2)
df['temp_lag_7d'] = df['actual_temp'].shift(7)

# Rolling temperature features
df['temp_rolling_mean_7d'] = df['actual_temp'].rolling(window=7).mean()
df['temp_rolling_std_7d'] = df['actual_temp'].rolling(window=7).std()

# Temperature change rates
df['temp_change_1d'] = df['actual_temp'] - df['temp_lag_1d']
df['temp_change_rate'] = df['temp_change_1d'] / df['temp_lag_1d']


###Gas Demand Historical Features:
# Lagged demand features
df['demand_lag_1d'] = df['actual_gas_demand'].shift(1)
df['demand_lag_2d'] = df['actual_gas_demand'].shift(2)
df['demand_lag_7d'] = df['actual_gas_demand'].shift(7)
df['demand_lag_30d'] = df['actual_gas_demand'].shift(30)

# Rolling demand statistics
df['demand_rolling_mean_7d'] = df['actual_gas_demand'].rolling(window=7).mean()
df['demand_rolling_std_7d'] = df['actual_gas_demand'].rolling(window=7).std()
df['demand_rolling_max_7d'] = df['actual_gas_demand'].rolling(window=7).max()


###HDD and Wind-Related Features:
# HDD features
df['hdd_lag_1d'] = df['HDD'].shift(1)
df['hdd_rolling_mean_7d'] = df['HDD'].rolling(window=7).mean()

# Wind features
df['wind_lag_1d'] = df['avg_wind'].shift(1)
df['wind_rolling_mean_7d'] = df['avg_wind'].rolling(window=7).mean()

# Interaction features
df['hdd_wind_interaction'] = df['HDD'] * df['avg_wind']


###Forecast Error Features:
# Demand forecast error features
df['demand_forecast_error'] = df['actual_gas_demand'] - df['forecasted_gas_demand']
df['demand_forecast_error_pct'] = df['demand_forecast_error'] / df['forecasted_gas_demand']

# Temperature forecast error features
df['temp_forecast_error'] = df['actual_temp'] - df['forecasted_average_temp']
df['temp_forecast_error_pct'] = df['temp_forecast_error'] / df['forecasted_average_temp']

###Cyclical Features (to better capture periodicity):
# Cyclical encoding of time features
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)


###Additional Interaction Features:
# Create interaction features between important variables
df['temp_hdd_interaction'] = df['actual_temp'] * df['HDD']
df['wind_temp_interaction'] = df['avg_wind'] * df['actual_temp']
df['month_hdd_interaction'] = df['month'] * df['HDD']

### Important considerations:

- Handle missing values created by lag features appropriately
- Scale or normalize features if needed (though tree-based models like XGBoost and LightGBM can handle different scales)
- Consider removing highly correlated features
- Use feature importance from initial models to select most relevant features

In [ ]:
# Forward fill for time series data
df = df.fillna(method='ffill')

# Or drop rows with missing values if at the start of the dataset
df = df.dropna()

In [ ]:
#Feature selection using correlation:
def remove_highly_correlated_features(df, threshold=0.95, method='pearson'):
    # Calculate correlation matrix
    corr_matrix = df.corr(method=method).abs()
    
    # Create upper triangle matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features to drop
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    print(f"Features to drop: {to_drop}")
    return df.drop(columns=to_drop)

# 사용 예시:
df_features = remove_highly_correlated_features(df, threshold=0.95, method='pearson')

In [ ]:
def remove_highly_correlated_features(df, threshold=0.95, method='pearson'):
    """
    Remove highly correlated features and return both cleaned dataframe and list of removed features
    
    Parameters:
    df (pandas.DataFrame): Input dataframe
    threshold (float): Correlation threshold for removal (default: 0.95)
    method (str): Correlation method ('pearson', 'spearman', or 'kendall')
    
    Returns:
    tuple: (cleaned_df, dropped_features, correlation_details)
        - cleaned_df: DataFrame with highly correlated features removed
        - dropped_features: List of removed feature names
        - correlation_details: DataFrame containing details of dropped correlations
    """
    # Calculate correlation matrix
    corr_matrix = df.corr(method=method).abs()
    
    # Create upper triangle matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Store correlation details
    correlation_details = []
    
    # Find features to drop
    to_drop = []
    for column in upper.columns:
        # Find highly correlated pairs
        high_corr = upper[column][upper[column] > threshold]
        
        if len(high_corr) > 0:
            if column not in to_drop:
                to_drop.append(column)
                
            # Store correlation details
            for idx, corr in high_corr.items():
                correlation_details.append({
                    'dropped_feature': column,
                    'correlated_with': idx,
                    'correlation': corr
                })
    
    # Convert correlation details to DataFrame
    correlation_details = pd.DataFrame(correlation_details)
    
    # Remove features
    cleaned_df = df.drop(columns=to_drop)
    
    return cleaned_df, to_drop, correlation_details

# 사용 예시:
cleaned_df, dropped_features, corr_details = remove_highly_correlated_features(df, threshold=0.95, method='pearson')

# 결과 확인
print("\nDropped Features:", dropped_features)
print("\nCorrelation Details:")
print(corr_details)

## XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_mpae(y_true, y_pred, epsilon=0.5):
   """
   Calculate Mean Percentage Absolute Error with epsilon adjustment for zero values
   """
   y_true = np.array(y_true)
   y_pred = np.array(y_pred)
   
   # Add epsilon to zero values
   y_true = np.where(y_true == 0, epsilon, y_true)
   
   # Calculate percentage absolute error
   pae = np.abs((y_true - y_pred) / y_true) * 100
   
   return np.mean(pae)

def train_evaluate_xgboost(X, y, random_state=42):
   """
   Train and evaluate XGBoost model
   """
   # Split data into training and testing sets
   X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=0.2, shuffle=False  # No shuffle for time series data
   )
   
   # Initialize XGBoost model
   xgb_model = xgb.XGBRegressor(
       n_estimators=1000,
       learning_rate=0.01,
       max_depth=7,
       min_child_weight=1,
       subsample=0.8,
       colsample_bytree=0.8,
       random_state=random_state,
       n_jobs=-1
   )
   
   # Train model with early stopping
   eval_set = [(X_train, y_train), (X_test, y_test)]
   xgb_model.fit(
       X_train, y_train,
       eval_set=eval_set,
       eval_metric='rmse',
       early_stopping_rounds=50,
       verbose=100
   )
   
   # Make predictions
   y_pred = xgb_model.predict(X_test)
   
   # Calculate metrics
   metrics = {
       'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
       'mae': mean_absolute_error(y_test, y_pred),
       'r2': r2_score(y_test, y_pred),
       'mpae': calculate_mpae(y_test, y_pred, epsilon=0.5)
   }
   
   # Get feature importance
   feature_importance = pd.DataFrame({
       'feature': X_train.columns,
       'importance': xgb_model.feature_importances_
   }).sort_values('importance', ascending=False)
   
   # Print model performance
   print("\n=== Model Performance ===")
   print(f"RMSE: {metrics['rmse']:.4f}")
   print(f"MAE: {metrics['mae']:.4f}")
   print(f"R2: {metrics['r2']:.4f}")
   print(f"MPAE: {metrics['mpae']:.4f}%")
   
   # Plot feature importance
   plt.figure(figsize=(10, 6))
   sns.barplot(
       data=feature_importance.head(10),
       x='importance',
       y='feature'
   )
   plt.title('XGBoost Top 10 Feature Importance')
   plt.tight_layout()
   plt.show()
   
   # Plot actual vs predicted
   plt.figure(figsize=(10, 6))
   plt.scatter(y_test, y_pred, alpha=0.5)
   plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
   plt.xlabel('Actual')
   plt.ylabel('Predicted')
   plt.title('XGBoost: Actual vs Predicted')
   plt.tight_layout()
   plt.show()
   
   results = {
       'model': xgb_model,
       'predictions': y_pred,
       'metrics': metrics,
       'feature_importance': feature_importance,
       'test_index': X_test.index
   }
   
   return results

# 하이퍼파라미터 튜닝을 위한 함수
def tune_xgboost(X_train, y_train):
   """
   Tune XGBoost hyperparameters using GridSearchCV
   """
   param_grid = {
       'max_depth': [3, 5, 7],
       'learning_rate': [0.01, 0.1],
       'n_estimators': [100, 500, 1000],
       'min_child_weight': [1, 3, 5],
       'subsample': [0.6, 0.8, 1.0],
       'colsample_bytree': [0.6, 0.8, 1.0]
   }
   
   xgb_model = xgb.XGBRegressor(random_state=42)
   grid_search = GridSearchCV(
       estimator=xgb_model,
       param_grid=param_grid,
       cv=TimeSeriesSplit(n_splits=5),
       scoring='neg_root_mean_squared_error',
       n_jobs=-1,
       verbose=2
   )
   
   grid_search.fit(X_train, y_train)
   print("\nBest parameters:", grid_search.best_params_)
   return grid_search.best_params_

# 모델 실행
def run_xgboost_modeling(df, target_col, feature_cols):
   """
   Run complete XGBoost modeling process
   """
   # Prepare data
   X = df[feature_cols]
   y = df[target_col]
   
   # Train and evaluate model
   results = train_evaluate_xgboost(X, y)
   
   return results

# 사용 예시:
# feature_cols = [col for col in df.columns if col != target_col]
# results = run_xgboost_modeling(df, target_col='actual_gas_demand', feature_cols=feature_cols)

# 모델 저장
# results['model'].save_model('xgb_model.json')

## Light GBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_mpae(y_true, y_pred, epsilon=0.5):
   """
   Calculate Mean Percentage Absolute Error with epsilon adjustment for zero values
   """
   y_true = np.array(y_true)
   y_pred = np.array(y_pred)
   
   # Add epsilon to zero values
   y_true = np.where(y_true == 0, epsilon, y_true)
   
   # Calculate percentage absolute error
   pae = np.abs((y_true - y_pred) / y_true) * 100
   
   return np.mean(pae)

def train_evaluate_lightgbm(X, y, random_state=42):
   """
   Train and evaluate LightGBM model
   """
   # Split data into training and testing sets
   X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=0.2, shuffle=False  # No shuffle for time series data
   )
   
   # Initialize LightGBM model
   lgb_model = lgb.LGBMRegressor(
       n_estimators=1000,
       learning_rate=0.01,
       num_leaves=31,
       subsample=0.8,
       colsample_bytree=0.8,
       random_state=random_state,
       n_jobs=-1
   )
   
   # Train model with early stopping
   eval_set = [(X_test, y_test)]
   lgb_model.fit(
       X_train, y_train,
       eval_set=eval_set,
       eval_metric='rmse',
       early_stopping_rounds=50,
       verbose=100
   )
   
   # Make predictions
   y_pred = lgb_model.predict(X_test)
   
   # Calculate metrics
   metrics = {
       'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
       'mae': mean_absolute_error(y_test, y_pred),
       'r2': r2_score(y_test, y_pred),
       'mpae': calculate_mpae(y_test, y_pred, epsilon=0.5)
   }
   
   # Get feature importance
   feature_importance = pd.DataFrame({
       'feature': X_train.columns,
       'importance': lgb_model.feature_importances_
   }).sort_values('importance', ascending=False)
   
   # Print model performance
   print("\n=== Model Performance ===")
   print(f"RMSE: {metrics['rmse']:.4f}")
   print(f"MAE: {metrics['mae']:.4f}")
   print(f"R2: {metrics['r2']:.4f}")
   print(f"MPAE: {metrics['mpae']:.4f}%")
   
   # Plot feature importance
   plt.figure(figsize=(10, 6))
   sns.barplot(
       data=feature_importance.head(10),
       x='importance',
       y='feature'
   )
   plt.title('LightGBM Top 10 Feature Importance')
   plt.tight_layout()
   plt.show()
   
   # Plot actual vs predicted
   plt.figure(figsize=(10, 6))
   plt.scatter(y_test, y_pred, alpha=0.5)
   plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
   plt.xlabel('Actual')
   plt.ylabel('Predicted')
   plt.title('LightGBM: Actual vs Predicted')
   plt.tight_layout()
   plt.show()
   
   results = {
       'model': lgb_model,
       'predictions': y_pred,
       'metrics': metrics,
       'feature_importance': feature_importance,
       'test_index': X_test.index
   }
   
   return results

# 하이퍼파라미터 튜닝을 위한 함수
def tune_lightgbm(X_train, y_train):
   """
   Tune LightGBM hyperparameters using GridSearchCV
   """
   param_grid = {
       'num_leaves': [31, 62, 127],
       'learning_rate': [0.01, 0.1],
       'n_estimators': [100, 500, 1000],
       'subsample': [0.6, 0.8, 1.0],
       'colsample_bytree': [0.6, 0.8, 1.0],
       'min_child_samples': [20, 50, 100]
   }
   
   lgb_model = lgb.LGBMRegressor(random_state=42)
   grid_search = GridSearchCV(
       estimator=lgb_model,
       param_grid=param_grid,
       cv=TimeSeriesSplit(n_splits=5),
       scoring='neg_root_mean_squared_error',
       n_jobs=-1,
       verbose=2
   )
   
   grid_search.fit(X_train, y_train)
   print("\nBest parameters:", grid_search.best_params_)
   return grid_search.best_params_

# 모델 실행
def run_lightgbm_modeling(df, target_col, feature_cols):
   """
   Run complete LightGBM modeling process
   """
   # Prepare data
   X = df[feature_cols]
   y = df[target_col]
   
   # Train and evaluate model
   results = train_evaluate_lightgbm(X, y)
   
   return results

# 사용 예시:
# feature_cols = [col for col in df.columns if col != target_col]
# results = run_lightgbm_modeling(df, target_col='actual_gas_demand', feature_cols=feature_cols)

# 모델 저장
# results['model'].save_model('lgb_model.txt')

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def train_evaluate_models(X, y, random_state=42):
    """
    Train and evaluate XGBoost and LightGBM models
    
    Parameters:
    X (pd.DataFrame): Feature matrix
    y (pd.Series): Target variable
    random_state (int): Random seed for reproducibility
    
    Returns:
    dict: Dictionary containing trained models and their performance metrics
    """
    # Time series split for validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False  # No shuffle for time series data
    )
    
    # Initialize models
    xgb_model = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=7,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=random_state,
        n_jobs=-1
    )
    
    lgb_model = lgb.LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.01,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=random_state,
        n_jobs=-1
    )
    
    # Train models with early stopping
    eval_set_xgb = [(X_train, y_train), (X_test, y_test)]
    eval_set_lgb = [(X_test, y_test)]
    
    # Train XGBoost
    xgb_model.fit(
        X_train, y_train,
        eval_set=eval_set_xgb,
        eval_metric='rmse',
        early_stopping_rounds=50,
        verbose=100
    )
    
    # Train LightGBM
    lgb_model.fit(
        X_train, y_train,
        eval_set=eval_set_lgb,
        eval_metric='rmse',
        early_stopping_rounds=50,
        verbose=100
    )
    
    # Make predictions
    xgb_pred = xgb_model.predict(X_test)
    lgb_pred = lgb_model.predict(X_test)
    
    # Calculate metrics
    def calculate_metrics(y_true, y_pred, model_name):
        return {
            f'{model_name}_rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            f'{model_name}_mae': mean_absolute_error(y_true, y_pred),
            f'{model_name}_r2': r2_score(y_true, y_pred)
        }
    
    # Get metrics for both models
    xgb_metrics = calculate_metrics(y_test, xgb_pred, 'xgb')
    lgb_metrics = calculate_metrics(y_test, lgb_pred, 'lgb')
    
    # Get feature importance
    xgb_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    lgb_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': lgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Combine all results
    results = {
        'xgb_model': xgb_model,
        'lgb_model': lgb_model,
        'xgb_predictions': xgb_pred,
        'lgb_predictions': lgb_pred,
        'metrics': {**xgb_metrics, **lgb_metrics},
        'feature_importance': {
            'xgb': xgb_importance,
            'lgb': lgb_importance
        },
        'test_index': X_test.index
    }
    
    return results

# 모델 학습 및 평가 실행
def run_modeling(df, target_col, feature_cols):
    """
    Run the complete modeling process
    
    Parameters:
    df (pd.DataFrame): Complete dataset
    target_col (str): Name of target column
    feature_cols (list): List of feature column names
    """
    # Prepare data
    X = df[feature_cols]
    y = df[target_col]
    
    # Train and evaluate models
    results = train_evaluate_models(X, y)
    
    # Print results
    print("\n=== Model Performance ===")
    print("\nXGBoost Metrics:")
    print(f"RMSE: {results['metrics']['xgb_rmse']:.4f}")
    print(f"MAE: {results['metrics']['xgb_mae']:.4f}")
    print(f"R2: {results['metrics']['xgb_r2']:.4f}")
    
    print("\nLightGBM Metrics:")
    print(f"RMSE: {results['metrics']['lgb_rmse']:.4f}")
    print(f"MAE: {results['metrics']['lgb_mae']:.4f}")
    print(f"R2: {results['metrics']['lgb_r2']:.4f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.barplot(
        data=results['feature_importance']['xgb'].head(10),
        x='importance',
        y='feature'
    )
    plt.title('XGBoost Top 10 Feature Importance')
    
    plt.subplot(1, 2, 2)
    sns.barplot(
        data=results['feature_importance']['lgb'].head(10),
        x='importance',
        y='feature'
    )
    plt.title('LightGBM Top 10 Feature Importance')
    plt.tight_layout()
    plt.show()
    
    # Plot actual vs predicted
    plt.figure(figsize=(15, 6))
    
    plt.subplot(1, 2, 1)
    plt.scatter(y[results['test_index']], results['xgb_predictions'], alpha=0.5)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('XGBoost: Actual vs Predicted')
    
    plt.subplot(1, 2, 2)
    plt.scatter(y[results['test_index']], results['lgb_predictions'], alpha=0.5)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('LightGBM: Actual vs Predicted')
    plt.tight_layout()
    plt.show()
    
    return results

# 사용 예시:
# feature_cols = [col for col in df.columns if col != target_col]
# results = run_modeling(df, target_col='actual_gas_demand', feature_cols=feature_cols)

# 모델 저장
# results['xgb_model'].save_model('xgb_model.json')
# results['lgb_model'].save_model('lgb_model.txt')

In [ ]:
from sklearn.model_selection import GridSearchCV

def tune_xgboost(X_train, y_train):
    param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'n_estimators': [100, 500, 1000],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }
    
    xgb_model = xgb.XGBRegressor(random_state=42)
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        cv=TimeSeriesSplit(n_splits=5),
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        verbose=2
    )
    
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_